In [3]:
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
import nest_asyncio
import os

nest_asyncio.apply()

# --- API Setup ---
API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

# --- Async Fetch Function ---
async def fetch_options_for_ticker(session, ticker, delta_range, theta_range, min_exp, max_exp, min_open_interest):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": ticker,
        "page[limit]": 1000
    }

    try:
        async with session.get(BASE_URL, params=params, timeout=10) as response:
            response.raise_for_status()
            json_response = await response.json()
            contracts = json_response.get("data", [])

            results = []
            for contract in contracts:
                attr = contract.get("attributes", {})
                try:
                    exp_date = datetime.strptime(attr.get("exp_date", ""), "%Y-%m-%d")
                    delta = float(attr.get("delta", 0))
                    theta = float(attr.get("theta", 0))
                    open_interest = int(attr.get("open_interest", 0))
                except:
                    continue

                if not (min_exp <= exp_date <= max_exp):
                    continue
                if not (delta_range[0] <= delta <= delta_range[1]):
                    continue
                if not (theta_range[0] <= theta <= theta_range[1]):
                    continue
                if open_interest < min_open_interest:
                    continue

                results.append({
                    "Ticker": ticker,
                    "Expiration": exp_date.date(),
                    "Strike": attr.get("strike"),
                    "Type": attr.get("type"),
                    "Delta": delta,
                    "Theta": theta,
                    "IV": attr.get("volatility"),
                    "Last": attr.get("last"),
                    "Open Interest": open_interest,
                    "Bid": attr.get("bid"),
                    "Bid Date": attr.get("bid_date"),
                    "Ask": attr.get("ask"),
                    "Ask Date": attr.get("ask_date"),
                    "Volume": attr.get("volume")
                })

            return results
    except Exception as e:
        print(f"Error fetching for {ticker}: {e}")
        return []

# --- Main Async Aggregator ---
async def fetch_filtered_options_async(
    tickers,
    delta_range=(0.6, 1),
    theta_range=(-0.05, -0.0),
    days_range=(90, 500),
    buffer_days=60,
    min_open_interest=100
):
    all_results = []
    today = datetime.today()
    min_exp = today + timedelta(days=days_range[0] - buffer_days)
    max_exp = today + timedelta(days=days_range[1] + buffer_days)

    async with aiohttp.ClientSession() as session:
        tasks = [
            fetch_options_for_ticker(session, ticker, delta_range, theta_range, min_exp, max_exp, min_open_interest)
            for ticker in tickers
        ]
        all_data = await asyncio.gather(*tasks)
        for result in all_data:
            all_results.extend(result)

    return pd.DataFrame(all_results)

# --- Define Your Inputs ---
tickers = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'AMD']

params = {
    "delta_range": (0.6, 1),
    "theta_range": (-0.05, 0.0),
    "days_range": (270, 550),
    "buffer_days": 30,
    "min_open_interest": 100
}

# --- Run & Save Output ---
filtered_df = asyncio.get_event_loop().run_until_complete(
    fetch_filtered_options_async(tickers, **params)
)

# Display in console
print(filtered_df)

# Save as CSV to current directory
csv_filename = "theta_delta.csv"
filtered_df.to_csv(csv_filename, index=False)

# Print full path to confirm location
full_path = os.path.abspath(csv_filename)
print(f"\nSaved {len(filtered_df)} rows to CSV at:\n{full_path}")


Error fetching for NVDA: 
    Ticker  Expiration  Strike  Type     Delta     Theta      IV  Premium  \
0     AAPL  2026-03-20     130  call  0.924672 -0.027904  0.4175    77.85   
1     AAPL  2026-03-20     135  call  0.912310 -0.030120  0.4108    71.50   
2     AAPL  2026-03-20     140  call  0.906310 -0.030314  0.3852    70.05   
3     AAPL  2026-03-20     145  call  0.890542 -0.032668  0.3807    64.20   
4     AAPL  2026-03-20     150  call  0.875539 -0.034394  0.3709    60.10   
..     ...         ...     ...   ...       ...       ...     ...      ...   
191    AMD  2026-12-18     105  call  0.728777 -0.025968  0.4674    35.50   
192    AMD  2026-12-18     110  call  0.701402 -0.026642  0.4630    32.40   
193    AMD  2026-12-18     115  call  0.673688 -0.027193  0.4595    31.50   
194    AMD  2026-12-18     120  call  0.645918 -0.027627  0.4568    29.10   
195    AMD  2026-12-18     125  call  0.618576 -0.027989  0.4558    26.90   

     Open Interest    Bid             Bid Date   